In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
import sqlite3
from datetime import datetime
import matplotlib.pyplot as plt
from collections import defaultdict
# from main_utils import *
from utils import *
import time
MODEL = "best.pt"
model = YOLO(MODEL)

# cap = cap = cv2.VideoCapture(0)

cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
heatmap_obj = heatmap.Heatmap()
heatmap_obj.set_args(colormap=cv2.COLORMAP_PARULA, imw=frame_width, imh=frame_height, shape="circle")

detections = []
count = 0
track_history = defaultdict(lambda: [])

conn = sqlite3.connect('detection_database.db')
cursor = conn.cursor()

# Create detection table if it does not exit
create_detection_table(cursor)

# Function to update the Matplotlib plot with new frames
def update_plot(frame):
    cv2.imshow('output',frame)

plt.ion()  # Turn on interactive mode for Matplotlib
#[{'time': '2024-02-23 14:20:37', 'xyxy': [[28.64773178100586, 134.34608459472656, 640.0, 479.8232727050781]], 'confidence': [0.893378496170044], 'class_id': [0], 'object_id': [1]}]
try:
    while cap.isOpened():
        success, image = cap.read()
        if success:
            results = model.track(image, persist=True, tracker="bytetrack.yaml")
            heatmap_frame = heatmap_obj.generate_heatmap(image.copy(), results)
            update_plot(heatmap_frame)
            if results[0].boxes.id is not None:
                detections = [{'time': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                               'xyxy': list(results[0].boxes.xyxy.cpu().tolist()),
                               'confidence': list(results[0].boxes.conf.cpu().tolist()),
                               'class_id': list(results[0].boxes.cls.cpu().int().tolist()),
                               'object_id': list(map(int, list(results[0].boxes.id.cpu().tolist())))}]
                frame = results[0].plot()
                annotated_frame = plot_tracks(frame, results, track_history)
                try:
                    insert_detection(cursor, detections)
                    conn.commit()  # Commit the transaction
                except Exception as e:
                    print("Error inserting data into database:", e)
                    conn.rollback()  # Rollback the transaction in case of an error
            else:
                annotated_frame = None

            if annotated_frame is not None and annotated_frame.size > 0:
              
                # update_plot(annotated_frame)
                pass
            else:
              pass
                # update_plot(image)

            plt.pause(0.5)

        else:
            break

except KeyboardInterrupt:
    pass
  

cap.release()
cv2.destroyAllWindows()


In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
!pip install --upgrade pip setuptools

In [ ]:
!pip install tensorflow-gpu

In [ ]:
import cv2

print("OpenCV version:", cv2.__version__)

In [2]:
from ultralytics import YOLO
model = YOLO("best.pt")
model.names

{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'bus', 5: 'truck'}

In [43]:
import sqlite3
conn = sqlite3.connect('detection_database.db')
cursor = conn.cursor()
command = "SELECT * FROM detection_bytime WHERE time>'2024-02-24 00:00:00'"
# Execute a query to fetch data from the database
cursor.execute(command)
data = cursor.fetchall()

# Close the database cursor and connection
cursor.close()
conn.close()
print(len(data))

378


In [18]:
data
classes = {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 
            4: 'bus', 5: 'truck'}

In [33]:
for arr in data[:10]:
    dtime = arr[1]
    count = {i:0 for i, _ in classes.items()}
    for c in ast.literal_eval(arr[4]):
        count[c] += 1
    
    print(dtime, count)

2024-02-24 11:24:26 {0: 2, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0}
2024-02-24 11:24:28 {0: 2, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0}
2024-02-24 11:24:29 {0: 2, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0}
2024-02-24 11:24:30 {0: 2, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0}
2024-02-24 11:24:30 {0: 2, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0}
2024-02-24 11:24:31 {0: 2, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0}
2024-02-24 11:24:32 {0: 2, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0}
2024-02-24 11:24:33 {0: 2, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0}
2024-02-24 11:24:33 {0: 2, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0}
2024-02-24 11:31:22 {0: 1, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0}


In [34]:
data[:10]

[(1,
  '2024-02-24 11:24:26',
  '[[149.0539093017578, 312.4560546875, 455.0445251464844, 480.0], [473.79705810546875, 196.3007354736328, 639.57177734375, 480.0]]',
  '[0.703027069568634, 0.7056931257247925]',
  '[0, 0]',
  '[1, 2]'),
 (2,
  '2024-02-24 11:24:28',
  '[[136.6635284423828, 310.6073303222656, 446.42486572265625, 480.0], [474.2165832519531, 195.84947204589844, 640.0, 480.0]]',
  '[0.6713724732398987, 0.6848718523979187]',
  '[0, 0]',
  '[1, 2]'),
 (3,
  '2024-02-24 11:24:29',
  '[[127.39533233642578, 298.1336669921875, 459.6192321777344, 480.0], [480.4258117675781, 191.5514678955078, 640.0, 480.0]]',
  '[0.742521345615387, 0.4747643768787384]',
  '[0, 0]',
  '[1, 2]'),
 (4,
  '2024-02-24 11:24:30',
  '[[123.68619537353516, 297.05755615234375, 457.9034729003906, 480.0], [481.7348327636719, 188.5151824951172, 640.0, 480.0]]',
  '[0.7381100058555603, 0.4374709129333496]',
  '[0, 0]',
  '[1, 2]'),
 (5,
  '2024-02-24 11:24:30',
  '[[122.5447769165039, 296.80438232421875, 457.172